In [27]:
import pandas as pd
import numpy as np
import random
import math
import re


In [28]:
df = pd.read_csv("tourism_with_id.csv", encoding='ISO-8859-1')
df = df[df["City"] == "Jakarta"]
price_values = df['Price(RPH)'].tolist()

for i in range(len(price_values)):
  converted_amount = price_values[i] * 0.0054
  df['Price(RPH)'][i] = math.ceil(converted_amount)

df.rename(columns={'Price(RPH)': 'Price (INR)'}, inplace=True)

category_avg = df.groupby('Categories')['Time_Minutes'].mean()

# Replace NaN values with the average time taken for each category
df['Time_Minutes'] = df.apply(lambda row: category_avg[row['Categories']] if np.isnan(row['Time_Minutes']) else row['Time_Minutes'], axis=1)
is_nan_present_in_column = df['Time_Minutes'].isna().values.any()
df.loc[df['Categories'] == 'Islands', 'Time_Minutes'] = df.loc[df['Categories'] == 'Islands', 'Time_Minutes'].fillna(90)
df.loc[df['Categories'] == 'Religious', 'Time_Minutes'] = df.loc[df['Categories'] == 'Religious', 'Time_Minutes'].fillna(45)
is_nan_present_in_column = df['Time_Minutes'].isna().values.any()
df['Time_Minutes'] = df['Time_Minutes'].apply(math.ceil)
display(df)

final_df = df

C:\Users\prana\AppData\Local\Temp\ipykernel_29924\3893822602.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price(RPH)'][i] = math.ceil(converted_amount)


,Place_Id,Place_Name,Description,Categories,City,Price (INR),Ratings,Time_Minutes,Coordinate,lat,Long
0,1,National Monument,The National Monument or popularly abbreviated...,Culture,Jakarta,108,4.6,15,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153
1,2,Old town,"The old town in Jakarta, which is also called ...",Culture,Jakarta,0,4.6,90,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125
2,3,Fantasy world,Dunia Fantasi or also called Dufan is an enter...,Amusement park,Jakarta,1458,4.6,360,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538
3,4,Beautiful Indonesia in Miniature Park (TMII),Taman Mini Indonesia Indah is a tourist park w...,Amusement park,Jakarta,54,4.5,134,"{'lat': -6.30245899999999, 'lng': 106.8951559}",-6.302446,106.895156
4,5,Atlantis Water Adventure,"Atlantis Water Adventure, also known as Atlant...",Amusement park,Jakarta,508,4.5,60,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134
...,...,...,...,...,...,...,...,...,...,...,...
79,80,Plaza Indonesia,"Plaza Indonesia was inaugurated in early 1990,...",Shopping center,Jakarta,0,4.7,120,"{'lat': -6.193925900000002, 'lng': 106.8222158}",-6.193926,106.822216
80,81,Thamrin City Mall,Thamrin City or Thamrin City Mall is a large s...,Shopping center,Jakarta,0,4.4,60,"{'lat': -6.1946096, 'lng': 106.817905}",-6.194610,106.817905
81,82,The Satria Mandala Museum,The Satria Mandala Museum is a museum of the h...,Culture,Jakarta,27,4.5,150,"{'lat': -6.231698499999999, 'lng': 106.8185425}",-6.231699,106.818543
82,83,Alive Museum Ancol,Museums now don't only offer objects of histor...,Amusement park,Jakarta,1080,4.3,134,"{'lat': -6.117534399999999, 'lng': 106.857313}",-6.117534,106.857313


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 0 to 83
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Place_Id      84 non-null     int64  
 1   Place_Name    84 non-null     object 
 2   Description   84 non-null     object 
 3   Categories    84 non-null     object 
 4   City          84 non-null     object 
 5   Price (INR)   84 non-null     int64  
 6   Ratings       84 non-null     float64
 7   Time_Minutes  84 non-null     int64  
 8   Coordinate    84 non-null     object 
 9   lat           84 non-null     float64
 10  Long          84 non-null     float64
dtypes: float64(3), int64(3), object(5)
memory usage: 9.9+ KB


In [30]:
types = []
for i in df['Categories']:
  if i not in types:
    types.append(i)
  else:
    continue

print(types)

['Culture', 'Amusement park', 'Nature preserve', 'Islands', 'Shopping center', 'Religious']


In [31]:
max_budget = int(input("Enter the maximum budget of the tour, including accommodation; excluding travel within the city: "))
no_days = int(input("Enter the number of days of sightseeing: "))

In [32]:
hotel_df = pd.read_csv("hotels_jak.csv", encoding='ISO-8859-1')
hotel_df = hotel_df.drop_duplicates()
hotel_list = hotel_df['Name'].tolist()

def extract_coordinates_from_url(url):
    pattern = r"/data=!.*!3d(-?\d+\.\d+)!4d(-?\d+\.\d+)"
    match = re.search(pattern, url)

    if match:
        latitude = float(match.group(1))
        longitude = float(match.group(2))
        return latitude, longitude
    else:
        return None, None

display(hotel_df)

,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude
0,https://www.google.com/maps/place/Best+Western...,Best Western Senayan,4.5,3-star hotel,Casual hotel with dining & free Wi-Fi,2811,Free parking,Free breakfast,Free Wi-Fi,Air-conditioned,-6.228065,106.798835
1,https://www.google.com/maps/place/Mercure+Jaka...,Mercure Jakarta Gatot Subroto,4.6,4-star hotel,Upmarket hotel with dining & a spa,4535,Pool,Free parking,Free Wi-Fi,Air-conditioned,-6.239528,106.830420
2,https://www.google.com/maps/place/Ashley+Wahid...,Ashley Wahid Hasyim Jakarta,4.5,4-star hotel,Chic lodging with a modern restaurant,3918,Free parking,Free breakfast,Free Wi-Fi,Air-conditioned,-6.187497,106.827085
3,https://www.google.com/maps/place/HARRIS+Vertu...,HARRIS Vertu Hotel Harmoni Jakarta,4.6,4-star hotel,Cosmopolitan lodging with a spa,4007,Pool,Spa,Free parking,Free Wi-Fi,-6.164287,106.820651
4,https://www.google.com/maps/place/Hotel+Rivoli...,Hotel Rivoli Jakarta,4.2,3-star hotel,Straightforward hotel with dining,1785,Spa,Free parking,Free breakfast,Free Wi-Fi,-6.183943,106.844287
...,...,...,...,...,...,...,...,...,...,...,...,...
73,https://www.google.com/maps/place/Four+Points+...,"Four Points by Sheraton Jakarta, Thamrin",4.3,4-star hotel,Modern hotel with casual dining & a gym,4863,Spa,Free parking,Free Wi-Fi,Air-conditioned,-6.188306,106.822639
74,https://www.google.com/maps/place/K-Style+Eco+...,K-Style Eco Hotel Jakarta,3.9,2-star hotel,Modest budget hotel with a restaurant,862,Free parking,Air-conditioned,Wi-Fi,Restaurant,-6.159590,106.839515
75,https://www.google.com/maps/place/Hotel+Parago...,Hotel Paragon,4.0,3-star hotel,Casual lodging with a restaurant,1496,Free parking,Air-conditioned,Breakfast,Wi-Fi,-6.186664,106.831223
76,https://www.google.com/maps/place/THE+1O1+Jaka...,THE 1O1 Jakarta Sedayu Darmawangsa,4.5,4-star hotel,Striking hotel featuring city views,4632,Pool,Spa,Free parking,Free Wi-Fi,-6.253952,106.801820


In [33]:
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   ans = np.round(res, 2)
   return ans

def objective_function(solution, locations):
    total_distance = 0

    # Calculate the total distance for the solution
    for i in range(len(solution) - 1):
        current_location = solution[i]
        next_location = solution[i + 1]

        # Calculate the distance between locations
        distance = haversine_distance(locations[current_location], locations[next_location])
        total_distance += distance

    return total_distance


In [34]:
df1 = df[['Place_Name', 'lat', 'Long']]
df1.head(15)

,Place_Name,lat,Long
0,National Monument,-6.175392,106.827153
1,Old town,-6.137645,106.817125
2,Fantasy world,-6.125312,106.833538
3,Beautiful Indonesia in Miniature Park (TMII),-6.302446,106.895156
4,Atlantis Water Adventure,-6.124190,106.839134
5,Jaya Ancol Dreamland,-6.117333,106.857995
6,Ragunan Zoo,-6.312459,106.820187
7,Ocean Ecopark,-6.125802,106.836325
8,Harbor Marina,1.078880,103.931398
9,Tidung Island,-5.803205,106.523791


In [35]:
sorted_df = df[df['Ratings'] > 4]
new_df = sorted_df

categories = new_df['Categories'].unique()
print(categories)

user_categories = []
i=0
while i < len(categories):
  choice = input("Enter the categories you wish to visit (enter 0 to stop; enter 'all' to choose all categories): ")
  if choice == 'all':
    user_categories = categories
    break

  if (choice == '0'):
    break

  choice = choice.capitalize()
  if choice in categories:
    user_categories.append(choice)
    i=i+1

  else:
    print("Invalid entry")

print(user_categories)

user_categories_df = new_df[new_df['Categories'].isin(user_categories)]
df = user_categories_df
display(df)



['Culture' 'Amusement park' 'Nature preserve' 'Islands' 'Shopping center'
 'Religious']
['Culture' 'Amusement park' 'Nature preserve' 'Islands' 'Shopping center'
 'Religious']


,Place_Id,Place_Name,Description,Categories,City,Price (INR),Ratings,Time_Minutes,Coordinate,lat,Long
0,1,National Monument,The National Monument or popularly abbreviated...,Culture,Jakarta,108,4.6,15,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153
1,2,Old town,"The old town in Jakarta, which is also called ...",Culture,Jakarta,0,4.6,90,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125
2,3,Fantasy world,Dunia Fantasi or also called Dufan is an enter...,Amusement park,Jakarta,1458,4.6,360,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538
3,4,Beautiful Indonesia in Miniature Park (TMII),Taman Mini Indonesia Indah is a tourist park w...,Amusement park,Jakarta,54,4.5,134,"{'lat': -6.30245899999999, 'lng': 106.8951559}",-6.302446,106.895156
4,5,Atlantis Water Adventure,"Atlantis Water Adventure, also known as Atlant...",Amusement park,Jakarta,508,4.5,60,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134
...,...,...,...,...,...,...,...,...,...,...,...
79,80,Plaza Indonesia,"Plaza Indonesia was inaugurated in early 1990,...",Shopping center,Jakarta,0,4.7,120,"{'lat': -6.193925900000002, 'lng': 106.8222158}",-6.193926,106.822216
80,81,Thamrin City Mall,Thamrin City or Thamrin City Mall is a large s...,Shopping center,Jakarta,0,4.4,60,"{'lat': -6.1946096, 'lng': 106.817905}",-6.194610,106.817905
81,82,The Satria Mandala Museum,The Satria Mandala Museum is a museum of the h...,Culture,Jakarta,27,4.5,150,"{'lat': -6.231698499999999, 'lng': 106.8185425}",-6.231699,106.818543
82,83,Alive Museum Ancol,Museums now don't only offer objects of histor...,Amusement park,Jakarta,1080,4.3,134,"{'lat': -6.117534399999999, 'lng': 106.857313}",-6.117534,106.857313


In [36]:
max_tour_duration = 650  # Maximum tour duration in minutes
min_rating = 4.0
all_tours = []
TOURS = []

def get_location(place):
      lat = df.loc[df['Place_Name'] == place, 'lat'].item()
      long = df.loc[df['Place_Name'] == place, 'Long'].item()
      return lat, long

# Calculate the commute time based on the distance
def calculate_commute_time(distance):
      # Assuming a constant speed for commute
      speed = 30  # km/h
      commute_time = distance / speed  # in hours
      return (commute_time*60)

# Get the time spent at a place
def get_time_spent(place):
      time_spent = df.loc[df['Place_Name'] == place, 'Time_Minutes'].item()
      return time_spent

# Get the cost of visiting a place
def get_cost(place):
      cost = df.loc[df['Place_Name'] == place, 'Price (INR)'].item()
      return cost

# Get the rating of a place
def get_rating(place):
      rating = df.loc[df['Place_Name'] == place, 'Ratings'].item()
      return rating

# Calculate the total commute time in the tour
def calculate_total_commute_time(tour):
      total_commute_time = 0
      for i in range(len(tour)):
          place1 = tour[i]
          place2 = tour[(i + 1) % len(tour)]
          lat1, long1 = get_location(place1)
          lat2, long2 = get_location(place2)
          dist = haversine_distance(lat1, long1, lat2, long2)
          commute_time = calculate_commute_time(dist)
          total_commute_time += commute_time
      return total_commute_time


for i in range(3):
  for i in range(no_days):
    duration = []
    def initialize_population(num_moths, max_tour_duration, max_budget, min_rating):
      population = []
      attractions = df['Place_Name'].tolist()
      for _ in range(num_moths):
        tour = generate_random_tour(attractions, max_tour_duration, max_budget, min_rating)
        population.append(tour)


      return population


    def generate_random_tour(attractions, max_tour_duration, max_budget, min_rating):
      attractions = list(attractions)
      random.shuffle(attractions)
      tour = []
      distances = []
      tour_duration = 0

      if len(attractions) <= 1:
          return tour, distances

      for i in range(len(attractions)):
          attraction1 = attractions[i]

          if i < len(attractions) - 1:
              attraction2 = attractions[i + 1]

              lat1, long1 = get_location(attraction1)
              lat2, long2 = get_location(attraction2)
              dist = haversine_distance(lat1, long1, lat2, long2)

              commute_time = calculate_commute_time(dist)
              time_spent = get_time_spent(attraction2)

              # Check if adding the current attraction exceeds the max tour duration
              if tour_duration + commute_time + time_spent <= max_tour_duration:
                  distances.append([attraction1, attraction2, dist])
                  tour.append(attraction1)
                  tour_duration += commute_time + time_spent
              else:
                  # If adding the current attraction exceeds the max tour duration,
                  # calculate the remaining time available
                  remaining_time = max_tour_duration - tour_duration

                  # Check if the remaining time is enough for the commute time and time spent at the last attraction
                  if remaining_time >= (commute_time + get_time_spent(attractions[-1])):
                      # Add the last attraction to complete the tour
                      distances.append([attractions[-1], tour[0], dist])
                      tour.append(attractions[-1])
                      tour_duration += commute_time + get_time_spent(attractions[-1])
                      break  # Exit the loop since the tour is complete
                  else:
                      break  # Exit the loop since the remaining time is not enough

      # If the generated tour has less than 3 places, reshuffle and regenerate the tour

      if len(tour) < 3:
          return generate_random_tour(attractions, max_tour_duration, max_budget, min_rating)

      tour_duration = math.ceil(tour_duration)
      duration.append(tour_duration)

      return tour, distances

    tours = initialize_population(20, max_tour_duration, max_budget, min_rating)


    def calculate_tour_distance(moths):
      distances = []
      tour_list = []
      tour_distances = []
      for tour in moths:
        dists = tour[1]
        travelled_distance = 0
        for distance in dists:
          travelled_distance = travelled_distance + distance[2]
        distances.append(travelled_distance)
        tour_list.append(tour[0])

      tour_distances = [[x, y] for x, y in zip(tour_list, distances)]

      return tour_distances

    population_distances = calculate_tour_distance(tours)


    for i, j in zip(population_distances, duration):
      sum_ratings = 0
      average_rating = 0
      total_cost = 0
      for place in i[0]:
        rating_place = df.loc[df['Place_Name'] == place, 'Ratings'].iloc[0]
        cost = df.loc[df['Place_Name'] == place, 'Price (INR)'].iloc[0]
        sum_ratings = sum_ratings + rating_place
        total_cost = total_cost + cost

      average_rating = (sum_ratings) / len(i[0])
      i.append(round(average_rating, 2))
      i.append(total_cost)

      i.append(j)


    def evaluate_fitness(tours, max_tour_duration, max_budget, min_rating):
      fitness_values = []
      for tour, total_distance, total_rating, total_budget, total_duration in tours:
        if (total_duration <= max_tour_duration and total_budget <= max_budget and total_rating >= min_rating):
          fitness = calculate_fitness(total_distance, total_duration, total_budget, total_rating, max_budget, min_rating)
          fitness_values.append(fitness)
        else:
          fitness_values.append(0)  # Penalize tours violating the constraints
      return fitness_values


    def calculate_fitness(total_distance, total_duration, total_budget, total_rating, max_budget, min_rating):
      weight_distance = 1
      weight_duration = 3
      weight_budget = 2
      weight_rating = 4

      # Check if the budget constraint is violated
      if total_budget > max_budget:
        return 0  # Penalize the solution with a fitness value of 0

      # Check if the rating constraint is violated
      if total_rating < min_rating:
        return 0  # Penalize the solution with a fitness value of 0

      fitness = (
          weight_distance * (1 / total_distance)
          + weight_duration * (1 / total_duration)
          + weight_budget * ((max_budget - total_budget) / max_budget)  # Scale the budget fitness based on how close it is to the maximum budget
          + weight_rating * (total_rating - min_rating)  # Scale the rating fitness based on how far it is from the minimum rating
      )

      return fitness

    fitness = evaluate_fitness(population_distances, max_tour_duration, max_budget, min_rating)
    fitness_values = fitness


    def update_light_source(fitness, population_distances):
      best_fitness = max(fitness)
      best_tour = population_distances[fitness.index(best_fitness)][0]
      return best_tour, best_fitness

    def generate_permutations(tour, num_permutations):
      permutations = []
      for _ in range(num_permutations):
        permuted_tour = tour[:]
        random.shuffle(permuted_tour)
        permutations.append(permuted_tour)
      return permutations

    def moth_movement(moths, max_iterations, max_tour_duration, max_budget, min_rating):
      best_moth = None
      best_fitness = 0
      best_tour = []

      for iteration in range(max_iterations):
          # Evaluate fitness for each moth
          fitness = evaluate_fitness(moths, max_tour_duration, max_budget, min_rating)

          for moth, fit in zip(moths, fitness):
              # Update the best tour and fitness
              if fit > best_fitness:
                  best_moth = moth
                  best_fitness = fit
                  best_tour, best_fitness = update_light_source(fitness, moths)


          # Update the moth's position (tour) based on other moths
          other_moths = [m for m in moths if m != best_moth]
          random_moth = random.choice(other_moths)

          # Check if best_moth and random_moth have enough elements
          min_tour_length = min(len(best_moth), len(random_moth))
          for i in range(min_tour_length):
              fitness = evaluate_fitness(moths, max_tour_duration, max_budget, min_rating)
              if fitness[moths.index(random_moth)] > fitness[moths.index(best_moth)]:
                  if best_moth[i] != random_moth[i] and np.random.random() < 0.5:
                      best_moth[i] = random_moth[i]

      return best_moth


    best_tour = moth_movement(population_distances, 40, max_tour_duration, max_budget, min_rating)
    print("BEST TOUR = ",best_tour)
    all_tours.append(best_tour)
    

    sorted_fitness = sorted(enumerate(fitness_values), key=lambda x: x[1], reverse = True)
    indices = [index for index, _ in sorted_fitness]


    df = df[~df["Place_Name"].isin(best_tour[0])]

  df = user_categories_df

  my_tours = []
  for i in all_tours:
    my_tours.append(i[0])

  print("-----------------------------")

#tour, distance, rating, cost, duration

BEST TOUR =  [['Jakarta Aquarium and Safari', 'Rich Indonesia Gallery', 'National Awakening Museum', 'Tebet Forest Park'], 52.32, 4.6, 54, 647]
BEST TOUR =  [['Chinese Village', 'Basoeki Abdullah Museum', 'Beautiful Indonesia in Miniature Park (TMII)', 'National Museum', 'Cathedral Church', 'Pecenongan Culinary Tour'], 29.290000000000003, 4.64, 33, 633]
BEST TOUR =  [['Grand Indonesia Mall', 'BSM Culinary Area', 'National Monument', 'Inscription Park Museum', 'Golden Snail Legend Park', 'Puring Garden Market'], 47.69, 4.63, 281, 645]
BEST TOUR =  [['Indonesian National Gallery', 'Kidzania', 'Bull Field Park'], 14.79, 4.63, 567, 629]
-----------------------------
BEST TOUR =  [['Istiqlal Mosque', 'National Library', 'Ragunan Zoo'], 26.77, 4.63, 173, 631]
BEST TOUR =  [['Cathedral Church', 'Alive Museum Ancol', 'The Satria Mandala Museum', 'Bull Field Park'], 42.42, 4.58, 281, 623]
BEST TOUR =  [['Cattleya Park', 'Grand Indonesia Mall', 'Pecenongan Culinary Tour', 'Proclamation Monument'

In [37]:
for i in all_tours:
    print(i)

[['Jakarta Aquarium and Safari', 'Rich Indonesia Gallery', 'National Awakening Museum', 'Tebet Forest Park'], 52.32, 4.6, 54, 647]
[['Chinese Village', 'Basoeki Abdullah Museum', 'Beautiful Indonesia in Miniature Park (TMII)', 'National Museum', 'Cathedral Church', 'Pecenongan Culinary Tour'], 29.290000000000003, 4.64, 33, 633]
[['Grand Indonesia Mall', 'BSM Culinary Area', 'National Monument', 'Inscription Park Museum', 'Golden Snail Legend Park', 'Puring Garden Market'], 47.69, 4.63, 281, 645]
[['Indonesian National Gallery', 'Kidzania', 'Bull Field Park'], 14.79, 4.63, 567, 629]
[['Istiqlal Mosque', 'National Library', 'Ragunan Zoo'], 26.77, 4.63, 173, 631]
[['Cathedral Church', 'Alive Museum Ancol', 'The Satria Mandala Museum', 'Bull Field Park'], 42.42, 4.58, 281, 623]
[['Cattleya Park', 'Grand Indonesia Mall', 'Pecenongan Culinary Tour', 'Proclamation Monument', 'Stamp Museum', 'Spathodea Park'], 48.97, 4.63, 135, 569]
[['Indonesian National Gallery', 'Museum of Fine Arts and Cer

In [38]:
cost = 0
for i in all_tours:
  cost = cost + (7000*0.0054) + (i[1]-2)*4000*0.0054

for i in my_tours:
  for j in i:
    price = final_df.loc[final_df['Place_Name'] == j, 'Price (INR)'].iloc[0]
    cost = cost + price

cost_tours = math.ceil(cost)



In [39]:
budget_constraint = max_budget - cost_tours - 500

def remove_duplicates(lst):
    seen = set()
    result = []
    for item in lst:
        place = item[0]
        if place not in seen:
            seen.add(place)
            result.append(item)
    return result

def calculate_score(distance, rating, price_per_night, total_nights, budget):
    weight_distance = 2
    weight_rating = 5
    weight_budget = 3
    total_price = price_per_night * total_nights
    budget_difference = abs(budget - total_price)
    score = (weight_distance / (distance + 0.1)) + (weight_rating * rating) + (weight_budget / (budget_difference + 0.1))
    return score

hotels = hotel_df['Name'].tolist()
hotel_ratings = hotel_df['Rating'].tolist()
hotel_prices_per_night = hotel_df['Price per Night'].tolist()
total_nights = no_days + 2  # Example number of total nights, adjust according to your requirements

hotel_scores = []
for tour in my_tours:
    first_place = tour[0]
    first_place_lat = final_df.loc[final_df['Place_Name'] == first_place, 'lat'].iloc[0]
    first_place_long = final_df.loc[final_df['Place_Name'] == first_place, 'Long'].iloc[0]

    for hotel, rating, price_per_night in zip(hotels, hotel_ratings, hotel_prices_per_night):
        url = hotel_df.loc[hotel_df['Name'] == hotel, 'Maps'].iloc[0]
        hotel_latitude, hotel_longitude = extract_coordinates_from_url(url)
        distance = haversine_distance(first_place_lat, first_place_long, hotel_latitude, hotel_longitude)
        total_price = price_per_night * total_nights
        if total_price <= budget_constraint:
            score = calculate_score(distance, rating, price_per_night, total_nights, budget_constraint)
            hotel_scores.append((hotel, rating, distance, price_per_night, score))

hotel_scores.sort(key=lambda x: (x[1], abs(x[3] * total_nights - budget_constraint), x[2]), reverse=True)  # Sort by rating (descending), total price difference (ascending), and distance (ascending)
hotel_scores = remove_duplicates(hotel_scores)

top_hotels = [hotel for hotel, _, _, _, _ in hotel_scores]
print(top_hotels)




['Holiday Inn & Suites Jakarta Gajah Mada, an IHG Hotel', 'Hotel Ciputra Jakarta (Managed by Swiss-Belhotel International)', 'Hotel Des Indes, Menteng', 'Erian Hotel', 'Stanley Wahid Hasyim Jakarta', 'YELLO Hotel Manggarai - Jakarta', 'YELLO Harmoni Jakarta', 'Swiss-Belinn Wahid Hasyim', 'Liberty Hotel Thamrin Jakarta', 'Best Western Senayan', 'ARTOTEL Thamrin - Jakarta', 'Jambuluwuk Thamrin Hotel', 'Ashley Sabang Jakarta', 'Ashley Wahid Hasyim Jakarta', 'Sofyan Hotel Cut Meutia Cikini Menteng Jakarta', 'Sentral Cawang Hotel Jakarta', 'Ana Hotel', 'Grand Thamrin Hotel', 'ARTOTEL Wahid Hasyim - Jakarta', 'Eightin Hotel Sudirman Jakarta', 'Sotis Hotel Kemang Jakarta', 'Royal Kuningan Hotel', 'Best Western Premier The Hive', 'Harris Hotel Tebet - Jakarta', 'Lumire Hotel & Convention Center', 'Shakti', 'Cikini Hotel', 'PAX Hotel Jakarta', 'Hotel Marcopolo Jakarta', 'Park Hotel Jakarta', 'Ã©L Hotel Royale Jakarta', 'AONE Hotel Jakarta', 'Maple Hotel Grogol', 'P Hotel', 'Triniti Hotel Jakart

In [40]:
desired_order = []

for i in (top_hotels):
  index = hotel_df.index[hotel_df['Name'] == i].tolist()
  desired_order.append(index[0])

# Reorder the rows based on the desired order of row labels
hotel_df_reordered = hotel_df.reindex(desired_order)


In [41]:
top_hotels = [hotel for hotel, _, _, _, _ in hotel_scores]

dff = hotel_df_reordered

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


features = ['Rating', 'Type', 'Description', 'Price per Night', 'Amenities 1', 'Amenities 2', 'Amenities 3', 'Amenities 4']

dff['Features'] = dff[features[2:]].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(dff['Features'])

similarity_matrix = cosine_similarity(feature_vectors, feature_vectors)

def get_recommendations(keywords, top_n=5):

    user_input_doc = ' '.join(keywords)
    user_input_vector = vectorizer.transform([user_input_doc])

    similarity_scores = cosine_similarity(user_input_vector, feature_vectors)[0]

    top_indices = similarity_scores.argsort()[::-1][:top_n]
    top_hotels = dff.iloc[top_indices]

    top_hotels = top_hotels.sort_values(['Rating', 'Price per Night'], ascending=[False, True])

    return top_hotels['Name'].values

user_keywords = ['Free breakfast', 'Spa', 'Pool']
num_recommendations = 5
recommendations = get_recommendations(user_keywords, top_n=num_recommendations)

print("Recommendations based on distance, rating and budget:")

if len(top_hotels)>5:
  for i in range(5):
    print(f"{i+1}. {top_hotels[i]}")

  for hotel in top_hotels[:5]:
      recommended_hotels = dff[dff['Name'] == hotel]
      display(recommended_hotels)

else:
  for i in range(len(top_hotels)):
    print(f"{i+1}. {top_hotels[i]}")

  for hotel in top_hotels:
      recommended_hotels = dff[dff['Name'] == hotel]
      display(recommended_hotels)



print()
print(f"Recommendations based on user input keywords, rating, and price per night (Top {num_recommendations} Recommendations):")
for i, hotel in enumerate(recommendations):
    print(f"{i+1}. {hotel}")




for i, hotel in enumerate(recommendations):
    recommended_hotels = dff[dff['Name'] == hotel]
    display(recommended_hotels)




Recommendations based on distance, rating and budget:
1. Holiday Inn & Suites Jakarta Gajah Mada, an IHG Hotel
2. Hotel Ciputra Jakarta (Managed by Swiss-Belhotel International)
3. Hotel Des Indes, Menteng
4. Erian Hotel
5. Stanley Wahid Hasyim Jakarta


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
39,https://www.google.com/maps/place/Holiday+Inn+...,"Holiday Inn & Suites Jakarta Gajah Mada, an IH...",4.7,4-star hotel,Modern lodging with free Wi-Fi & dining,3527,Pool,Spa,Free parking,Free Wi-Fi,-6.145215,106.815086,Modern lodging with free Wi-Fi & dining 3527 P...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
37,https://www.google.com/maps/place/Hotel+Ciputr...,Hotel Ciputra Jakarta (Managed by Swiss-Belhot...,4.7,4-star hotel,Stylish hotel with a rooftop pool,3606,Pool,Spa,Free parking,Free Wi-Fi,-6.167883,106.786752,Stylish hotel with a rooftop pool 3606 Pool Sp...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
52,https://www.google.com/maps/place/Hotel+Des+In...,"Hotel Des Indes, Menteng",4.6,Hotel,Upscale lodging with dining & a bar,2601,Free parking,Free Wi-Fi,Air-conditioned,Breakfast,-6.197208,106.828358,Upscale lodging with dining & a bar 2601 Free ...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
30,https://www.google.com/maps/place/Erian+Hotel/...,Erian Hotel,4.6,3-star hotel,Relaxed lodging with a restaurant,2842,Free parking,Free Wi-Fi,Air-conditioned,Breakfast,-6.187469,106.829182,Relaxed lodging with a restaurant 2842 Free pa...


...

,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
24,https://www.google.com/maps/place/Sentral+Cawa...,Sentral Cawang Hotel Jakarta,4.4,3-star hotel,Contemporary hotel with dining & a pool,1754,Pool,Spa,Free parking,Free breakfast,-6.246978,106.872788,Contemporary hotel with dining & a pool 1754 P...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
25,https://www.google.com/maps/place/Best+Western...,Best Western Premier The Hive,4.4,4-star hotel,Modern lodging with a pool & dining,3074,Pool,Spa,Free parking,Free Wi-Fi,-6.244606,106.875471,Modern lodging with a pool & dining 3074 Pool ...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
59,https://www.google.com/maps/place/Lumire+Hotel...,Lumire Hotel & Convention Center,4.4,4-star hotel,Polished hotel with dining & a pool,3750,Pool,Spa,Free breakfast,Free Wi-Fi,-6.175975,106.840408,Polished hotel with dining & a pool 3750 Pool ...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
54,https://www.google.com/maps/place/%C3%A9L+Hote...,Ã©L Hotel Royale Jakarta,4.3,4-star hotel,Stylish lodging with a pool & a spa,2568,Pool,Spa,Free parking,Free breakfast,-6.15229,106.89624,Stylish lodging with a pool & a spa 2568 Pool ...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
12,https://www.google.com/maps/place/The+Jayakart...,The Jayakarta SP Jakarta Hotel & Spa,4.2,4-star hotel,Upscale lodging with dining & a pool,1829,Pool,Spa,Free parking,Free Wi-Fi,-6.147389,106.817227,Upscale lodging with dining & a pool 1829 Pool...


In [42]:
choice = 0
my_hotel = input("Enter the hotel of your choice: ")
if my_hotel in hotel_list:
  choice = 1

while choice != 1:
  print("Invalid entry")
  my_hotel = input("Enter the hotel of your choice: ")
  if my_hotel in hotel_list:
    choice = 1


In [43]:
def get_time_spent(place):
      time_spent = final_df.loc[final_df['Place_Name'] == place, 'Time_Minutes'].item()
      return time_spent

def get_location(place):
      lat = final_df.loc[final_df['Place_Name'] == place, 'lat'].item()
      long = final_df.loc[final_df['Place_Name'] == place, 'Long'].item()
      return lat, long

def get_distance_from_hotel(place, my_hotel):
  url = hotel_df.loc[hotel_df['Name'] == my_hotel, 'Maps'].item()
  hotel_lat, hotel_long = extract_coordinates_from_url(url)
  lat1, long1 = get_location(place)
  dist = haversine_distance(hotel_lat,hotel_long,lat1,long1)
  return dist

def generate_itinerary(places):
    itinerary = []
    current_time = 9 * 60  # Starting time in minutes (9 AM = 9 * 60 minutes)

    # Add hotel as the first place in the itinerary
    hotel = my_hotel

    for i in range(len(places)):
        place = places[i]
        time_spent = get_time_spent(place)

        if i == 0:
            commute_time = calculate_commute_time(get_distance_from_hotel(place, my_hotel))  # Calculate commute time from hotel
        else:
            previous_place = places[i - 1]
            lat1, long1 = get_location(previous_place)
            lat2, long2 = get_location(place)
            dist = haversine_distance(lat1, long1, lat2, long2)
            commute_time = calculate_commute_time(dist)

        start_time = current_time + commute_time
        end_time = start_time + time_spent

        itinerary.append((place, start_time, end_time))

        current_time = end_time

    return itinerary


In [44]:
TOURS = all_tours
list_total_cost = []
#print(TOURS)
TOTAL_COST = 0
hotel_price = hotel_df.loc[hotel_df['Name'] == my_hotel, 'Price per Night'].iloc[0]

print(f"Here's your complete itinerary 1: Hotel - {my_hotel} - Price per Night = Rs. {hotel_price}\n")
for i in range(no_days):
  lat = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "lat"].item()
  Long = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "Long"].item()
  hotel_index = hotel_list.index(my_hotel)
  hotel_lat, hotel_long = extract_coordinates_from_url(hotel_df.loc[hotel_index, 'Maps'])
  dist = haversine_distance(hotel_lat, hotel_long, lat, Long)

  lat2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "lat"].item()
  Long2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "Long"].item()
  dist2 = haversine_distance(hotel_lat, hotel_long, lat2, Long2)

  TOTAL_DISTANCE = TOURS[i][1] + dist + dist2
  TOTAL_DISTANCE = round(TOTAL_DISTANCE, 2)

  total_cab_cost = (7000*0.0054) + ((TOTAL_DISTANCE - 2)*4000*0.0054)
  total_cab_cost = math.ceil(total_cab_cost)

  print(f"Day {i+1} - ")
  print("Tour -",TOURS[i][0])
  print("Total distance (Starting from and ending at hotel) -",TOTAL_DISTANCE,"km")
  print("Rating - ",TOURS[i][2])
  print("Cost - Rs.",TOURS[i][3])
  print("Taxi cost - Rs.",total_cab_cost)
  print("----------------------------------")

  TOTAL_COST = TOTAL_COST + TOURS[i][3] + total_cab_cost



print("The cost for the whole trip is Rs.", TOTAL_COST)

print(f"Hotel cost = Rs. {(no_days+2)*hotel_price} for {no_days+2} nights.")
print(f"Total cost = Rs. {(no_days+2)*hotel_price + TOTAL_COST}")
list_total_cost.append((no_days+2)*hotel_price + TOTAL_COST)


Here's your complete itinerary 1: Hotel - Holiday Inn & Suites Jakarta Gajah Mada, an IHG Hotel - Price per Night = Rs. 3527

Day 1 - 
Tour - ['Jakarta Aquarium and Safari', 'Rich Indonesia Gallery', 'National Awakening Museum', 'Tebet Forest Park']
Total distance (Starting from and ending at hotel) - 67.66 km
Rating -  4.6
Cost - Rs. 54
Taxi cost - Rs. 1457
----------------------------------
Day 2 - 
Tour - ['Chinese Village', 'Basoeki Abdullah Museum', 'Beautiful Indonesia in Miniature Park (TMII)', 'National Museum', 'Cathedral Church', 'Pecenongan Culinary Tour']
Total distance (Starting from and ending at hotel) - 57.15 km
Rating -  4.64
Cost - Rs. 33
Taxi cost - Rs. 1230
----------------------------------
Day 3 - 
Tour - ['Grand Indonesia Mall', 'BSM Culinary Area', 'National Monument', 'Inscription Park Museum', 'Golden Snail Legend Park', 'Puring Garden Market']
Total distance (Starting from and ending at hotel) - 64.35 km
Rating -  4.63
Cost - Rs. 281
Taxi cost - Rs. 1385
----

In [45]:
print("Detailed Itinerary:")
print("Hotel: ",my_hotel)
print()
for i in all_tours[:4]:
    places = i[0]
    itinerary = generate_itinerary(places)

    print("Start from hotel at 09:00")
    for item in itinerary:
        place = item[0]
        start_time = divmod(item[1], 60)
        end_time = divmod(item[2], 60)  # Convert minutes to hours and minutes
        start_time_str = f"{int(start_time[0]):02}:{int(start_time[1]):02}"
        end_time_str = f"{int(end_time[0]):02}:{int(end_time[1]):02}"
        print(f"{start_time_str} to {end_time_str} - {place}")

    print("--------------------------------------------------------------")




Detailed Itinerary:
Hotel:  Holiday Inn & Suites Jakarta Gajah Mada, an IHG Hotel

Start from hotel at 09:00
09:08 to 11:22 - Jakarta Aquarium and Safari
11:30 to 13:00 - Rich Indonesia Gallery
13:05 to 13:50 - National Awakening Museum
14:04 to 15:04 - Tebet Forest Park
--------------------------------------------------------------
Start from hotel at 09:00
09:50 to 11:25 - Chinese Village
11:43 to 13:13 - Basoeki Abdullah Museum
13:36 to 15:50 - Beautiful Indonesia in Miniature Park (TMII)
16:22 to 17:57 - National Museum
18:00 to 18:45 - Cathedral Church
18:47 to 19:57 - Pecenongan Culinary Tour
--------------------------------------------------------------
Start from hotel at 09:00
09:11 to 11:11 - Grand Indonesia Mall
11:13 to 12:23 - BSM Culinary Area
12:25 to 12:40 - National Monument
12:42 to 14:12 - Inscription Park Museum
14:46 to 17:00 - Golden Snail Legend Park
17:27 to 18:27 - Puring Garden Market
--------------------------------------------------------------
Start from ho

In [46]:
for i in range(4):
  temp = all_tours.pop(0)

TOURS = all_tours

TOTAL_COST = 0
hotel_price = hotel_df.loc[hotel_df['Name'] == my_hotel, 'Price per Night'].iloc[0]

print(f"Here's your complete itinerary 2: Hotel - {my_hotel} - Price per Night = Rs. {hotel_price}\n")
for i in range(no_days):
  lat = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "lat"].item()
  Long = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "Long"].item()
  hotel_index = hotel_list.index(my_hotel)
  hotel_lat, hotel_long = extract_coordinates_from_url(hotel_df.loc[hotel_index, 'Maps'])
  dist = haversine_distance(hotel_lat, hotel_long, lat, Long)

  lat2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "lat"].item()
  Long2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "Long"].item()
  dist2 = haversine_distance(hotel_lat, hotel_long, lat2, Long2)

  TOTAL_DISTANCE = TOURS[i][1] + dist + dist2
  TOTAL_DISTANCE = round(TOTAL_DISTANCE, 2)

  total_cab_cost = (7000*0.0054) + ((TOTAL_DISTANCE - 2)*4000*0.0054)
  total_cab_cost = math.ceil(total_cab_cost)

  print(f"Day {i+1} - ")
  print("Tour -",TOURS[i][0])
  print("Total distance (Starting from and ending at hotel) -",TOTAL_DISTANCE,"km")
  print("Rating - ",TOURS[i][2])
  print("Cost - Rs.",TOURS[i][3])
  print("Taxi cost - Rs.",total_cab_cost)
  print("----------------------------------")

  TOTAL_COST = TOTAL_COST + TOURS[i][3] + total_cab_cost



print("The cost for the whole trip is Rs.", TOTAL_COST)

print(f"Hotel cost = Rs. {(no_days+2)*hotel_price} for {no_days+2} nights.")
print(f"Total cost = Rs. {(no_days+2)*hotel_price + TOTAL_COST}")
list_total_cost.append((no_days+2)*hotel_price + TOTAL_COST)


Here's your complete itinerary 2: Hotel - Holiday Inn & Suites Jakarta Gajah Mada, an IHG Hotel - Price per Night = Rs. 3527

Day 1 - 
Tour - ['Istiqlal Mosque', 'National Library', 'Ragunan Zoo']
Total distance (Starting from and ending at hotel) - 48.69 km
Rating -  4.63
Cost - Rs. 173
Taxi cost - Rs. 1047
----------------------------------
Day 2 - 
Tour - ['Cathedral Church', 'Alive Museum Ancol', 'The Satria Mandala Museum', 'Bull Field Park']
Total distance (Starting from and ending at hotel) - 49.33 km
Rating -  4.58
Cost - Rs. 281
Taxi cost - Rs. 1061
----------------------------------
Day 3 - 
Tour - ['Cattleya Park', 'Grand Indonesia Mall', 'Pecenongan Culinary Tour', 'Proclamation Monument', 'Stamp Museum', 'Spathodea Park']
Total distance (Starting from and ending at hotel) - 73.26 km
Rating -  4.63
Cost - Rs. 135
Taxi cost - Rs. 1578
----------------------------------
Day 4 - 
Tour - ['Indonesian National Gallery', 'Museum of Fine Arts and Ceramics', 'Welcome Monument', 'Yo

In [47]:
print("Detailed Itinerary:")
print("Hotel: ",my_hotel)
print()
for i in all_tours[:4]:
    places = i[0]
    itinerary = generate_itinerary(places)

    print("Start from hotel at 09:00")
    for item in itinerary:
        place = item[0]
        start_time = divmod(item[1], 60)
        end_time = divmod(item[2], 60)  # Convert minutes to hours and minutes
        start_time_str = f"{int(start_time[0]):02}:{int(start_time[1]):02}"
        end_time_str = f"{int(end_time[0]):02}:{int(end_time[1]):02}"
        print(f"{start_time_str} to {end_time_str} - {place}")

    print("--------------------------------------------------------------")




Detailed Itinerary:
Hotel:  Holiday Inn & Suites Jakarta Gajah Mada, an IHG Hotel

Start from hotel at 09:00
09:06 to 09:51 - Istiqlal Mosque
09:54 to 13:54 - National Library
14:23 to 14:38 - Ragunan Zoo
--------------------------------------------------------------
Start from hotel at 09:00
09:06 to 09:51 - Cathedral Church
10:04 to 12:18 - Alive Museum Ancol
12:45 to 15:15 - The Satria Mandala Museum
15:29 to 17:43 - Bull Field Park
--------------------------------------------------------------
Start from hotel at 09:00
09:09 to 11:23 - Cattleya Park
11:29 to 13:29 - Grand Indonesia Mall
13:35 to 14:45 - Pecenongan Culinary Tour
14:55 to 16:30 - Proclamation Monument
16:55 to 18:30 - Stamp Museum
18:48 to 19:18 - Spathodea Park
--------------------------------------------------------------
Start from hotel at 09:00
09:08 to 10:43 - Indonesian National Gallery
10:54 to 12:29 - Museum of Fine Arts and Ceramics
12:42 to 12:57 - Welcome Monument
13:02 to 14:37 - Youth Pledge Museum
14:5

In [48]:
for i in range(4):
  temp = all_tours.pop(0)

TOURS = all_tours

TOTAL_COST = 0
hotel_price = hotel_df.loc[hotel_df['Name'] == my_hotel, 'Price per Night'].iloc[0]

print(f"Here's your complete itinerary 3: Hotel - {my_hotel} - Price per Night = Rs. {hotel_price}\n")
for i in range(no_days):
  lat = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "lat"].item()
  Long = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "Long"].item()
  hotel_index = hotel_list.index(my_hotel)
  hotel_lat, hotel_long = extract_coordinates_from_url(hotel_df.loc[hotel_index, 'Maps'])
  dist = haversine_distance(hotel_lat, hotel_long, lat, Long)

  lat2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "lat"].item()
  Long2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "Long"].item()
  dist2 = haversine_distance(hotel_lat, hotel_long, lat2, Long2)

  TOTAL_DISTANCE = TOURS[i][1] + dist + dist2
  TOTAL_DISTANCE = round(TOTAL_DISTANCE, 2)

  total_cab_cost = (7000*0.0054) + ((TOTAL_DISTANCE - 2)*4000*0.0054)
  total_cab_cost = math.ceil(total_cab_cost)

  print(f"Day {i+1} - ")
  print("Tour -",TOURS[i][0])
  print("Total distance (Starting from and ending at hotel) -",TOTAL_DISTANCE,"km")
  print("Rating - ",TOURS[i][2])
  print("Cost - Rs.",TOURS[i][3])
  print("Taxi cost - Rs.",total_cab_cost)
  print("----------------------------------")

  TOTAL_COST = TOTAL_COST + TOURS[i][3] + total_cab_cost



print("The cost for the whole trip is Rs.", TOTAL_COST)

print(f"Hotel cost = Rs. {(no_days+2)*hotel_price} for {no_days+2} nights.")
print(f"Total cost = Rs. {(no_days+2)*hotel_price + TOTAL_COST}")
list_total_cost.append((no_days+2)*hotel_price + TOTAL_COST)


Here's your complete itinerary 3: Hotel - Holiday Inn & Suites Jakarta Gajah Mada, an IHG Hotel - Price per Night = Rs. 3527

Day 1 - 
Tour - ['Indonesian National Gallery', 'State Palace of the Republic of Indonesia', 'Youth Pledge Museum']
Total distance (Starting from and ending at hotel) - 68.26 km
Rating -  4.58
Cost - Rs. 11
Taxi cost - Rs. 1470
----------------------------------
Day 2 - 
Tour - ['Pecenongan Culinary Tour', 'BSM Culinary Area', 'Old town', 'Ancol Beach', 'Tebet Forest Park', 'Spathodea Park']
Total distance (Starting from and ending at hotel) - 71.94 km
Rating -  4.72
Cost - Rs. 864
Taxi cost - Rs. 1549
----------------------------------
Day 3 - 
Tour - ['Angel Island', 'National Museum', 'Stamp Museum', 'Ismail Marzuki Park', 'Cathedral Church']
Total distance (Starting from and ending at hotel) - 88.92 km
Rating -  4.72
Cost - Rs. 108
Taxi cost - Rs. 1916
----------------------------------
Day 4 - 
Tour - ['The Satria Mandala Museum', 'National Library', 'Ahmad

In [49]:
print("Detailed Itinerary:")
print("Hotel: ",my_hotel)
print()
for i in all_tours[:4]:
    places = i[0]
    itinerary = generate_itinerary(places)

    print("Start from hotel at 09:00")
    for item in itinerary:
        place = item[0]
        start_time = divmod(item[1], 60)
        end_time = divmod(item[2], 60)  # Convert minutes to hours and minutes
        start_time_str = f"{int(start_time[0]):02}:{int(start_time[1]):02}"
        end_time_str = f"{int(end_time[0]):02}:{int(end_time[1]):02}"
        print(f"{start_time_str} to {end_time_str} - {place}")

    print("--------------------------------------------------------------")




Detailed Itinerary:
Hotel:  Holiday Inn & Suites Jakarta Gajah Mada, an IHG Hotel

Start from hotel at 09:00
09:08 to 10:43 - Indonesian National Gallery
10:46 to 12:21 - State Palace of the Republic of Indonesia
12:26 to 14:01 - Youth Pledge Museum
--------------------------------------------------------------
Start from hotel at 09:00
09:05 to 10:15 - Pecenongan Culinary Tour
10:19 to 11:29 - BSM Culinary Area
11:39 to 13:09 - Old town
13:18 to 14:48 - Ancol Beach
15:14 to 16:14 - Tebet Forest Park
16:34 to 17:04 - Spathodea Park
--------------------------------------------------------------
Start from hotel at 09:00
09:28 to 10:58 - Angel Island
11:33 to 13:08 - National Museum
13:42 to 15:17 - Stamp Museum
15:46 to 17:21 - Ismail Marzuki Park
17:26 to 18:11 - Cathedral Church
--------------------------------------------------------------
Start from hotel at 09:00
09:19 to 11:49 - The Satria Mandala Museum
12:00 to 16:00 - National Library
16:06 to 16:51 - Ahmad Yani Sasmita Loka Mu

In [50]:
print(max_budget)
print(list_total_cost)

40000
[26651, 26223, 28980]


In [55]:
difference = []
for cost in list_total_cost:
    difference.append(max_budget-cost)

#print(difference)

percent_difference = []
for diff in difference:
    percent_difference.append(round(diff*100/max_budget,2))

print("Percentage difference between budget and total cost of the tours:")
print(percent_difference)


Percentage difference between budget and total cost of the tours:
[33.37, 34.44, 27.55]


In [69]:
rank = sorted(percent_difference, reverse=True)

#print(rank)

rank_list = []

for val in rank:
    ind = percent_difference.index(val)
    rank_list.append(ind+1)

for i,j in zip(rank_list, range(3)):
    print(f"Rank of Tour {j+1}: {i}")



Rank of Tour 1: 2
Rank of Tour 2: 1
Rank of Tour 3: 3
